In [2]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import AutoTokenizer

import os
import pandas as pd
from datasets import Dataset
from datasets import load_from_disk

In [3]:
print("GPU available:", torch.cuda.is_available())
!nvcc --version
torch.__version__


GPU available: True
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


'2.10.0+cu128'

### Configuration

In [4]:
pretrained_model_hf = "unsloth/Llama-3.2-3B-Instruct"
MAX_SEQ_LENGTH = 2048 # required for long news articles
load_in_4bit = True
dtype = None
model, _ = FastLanguageModel.from_pretrained(
    model_name= pretrained_model_hf,
    max_seq_length = MAX_SEQ_LENGTH, 
    dtype = torch.bfloat16,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

Memory footprint: 2.31 GB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_hf)

In [7]:
tokenizer.clean_up_tokenization_spaces = True
tokenizer.add_tokens(["<PAD>"])
tokenizer.pad_token = "<PAD>"
tokenizer.add_eos_token = False

model.resize_token_embeddings(new_num_tokens=len(tokenizer))
model.config.eos_token_id = tokenizer.eos_token_id

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


### LoRA Configuration

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    r = 32, 
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # saves lots of vram!!!
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
    max_seq_length = MAX_SEQ_LENGTH
)

Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [9]:
# Define a path to store the processed dataset
dataset_cache_path = "./summarization_dataset"

if os.path.exists(dataset_cache_path):
    print(f"Loading pre-processed dataset from {dataset_cache_path}...")
    dataset = load_from_disk(dataset_cache_path)

else:
    print("Creating dataset from scratch...")
    
    def create_dataset(directory):
        data_entries = []
        files = [f for f in os.listdir(directory) if f.endswith('.csv')]
        for filename in files:
            try:
                df = pd.read_csv(os.path.join(directory, filename), on_bad_lines='skip')
                article = " ".join(df['regular_string'].dropna().astype(str).tolist()).strip()
                summary = " ".join(df['selko_string'].dropna().astype(str).tolist()).strip()
                
                if len(article) > 50 and len(summary) > 10:
                    data_entries.append({"article": article, "summary": summary})
            except Exception as e:
                print(f"Skipping {filename}: {e}")
                continue
        return Dataset.from_list(data_entries)

    # Load raw data
    raw_dataset = create_dataset("./ylenews-fi-2014-2020-selko-par-sent-src/CSV")

    def format_to_gemma(examples):
        texts = []
        for article, summary in zip(examples['article'], examples['summary']):
            messages = [
                {"role": "user", "content": f"Tiivistä seuraava uutinen selkosuomeksi:\n\n{article}\n"},
                {"role": "model", "content": summary}
            ]
            text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
            texts.append(text)
        return { "text": texts }

    dataset = raw_dataset.map(format_to_gemma, batched=True)
    
    print(f"Saving dataset to {dataset_cache_path} for future runs...")
    dataset.save_to_disk(dataset_cache_path)

print(f"Dataset ready. Total rows: {len(dataset)}")

Loading pre-processed dataset from ./summarization_dataset...
Dataset ready. Total rows: 8261


In [10]:
print(dataset)

Dataset({
    features: ['article', 'summary', 'text'],
    num_rows: 8261
})


In [11]:
print("\n--- SAMPLE INPUT FOR TRAINING ---")
print(dataset[0]["text"])


--- SAMPLE INPUT FOR TRAINING ---
<bos><start_of_turn>user
Tiivistä seuraava uutinen selkosuomeksi:

Isopanda synnytti kaksoset Chengdun pandakeskuksessa maanantaina. Kaksoset olivat ensimmäiset tänä vuonna syntyneet. Kelin-niminen isopanda synnytti maanantaina tyttökaksoset Chengdun pandankasvatuskeskuksessa Kiinan lounaisosassa. Poikaset olivat maailman ensimmäiset tänä vuonna syntyneet pandakaksoset. Poikasten kerrotaan olevan hyvässä kunnossa. Kaksosista vanhempi painaa 118 grammaa ja nuorempi 70 grammaa. Poikaset syövät vain millilitran maitoa kerrallaan ruokaillessaan, kertoo hoitajia haastatellut Kiinan valtiollinen CCTV. Pandakeskuksen emopanda tuli raskaaksi keinosiemennyksestä. Isopanda on luokiteltu erittäin uhanalaiseksi lajiksi. WWF:n mukaan luonnossa elää vain 1 800 isopandaa.<end_of_turn>
<start_of_turn>model
Kiinassa on syntynyt vuoden ensimmäiset pandakaksoset. Pennut syntyivät maanantaina Chengdun kaupungin keskuksessa, jossa kasvatetaan pandakarhuja. Luonnossa elää 

In [ ]:
shuffled_dataset = dataset.shuffle(seed=3407)
dataset_small = shuffled_dataset.select(range(1000)) # remove if you want better model, training takes longer

In [16]:
from unsloth import UnslothTrainer, UnslothTrainingArguments
import math

batch_size = 4
eval_batch_size = 4
gradient_accumulation_steps = 16
epochs = 1
train_steps = math.ceil(len(dataset_small) / batch_size / gradient_accumulation_steps * epochs)
eval_steps = math.floor(train_steps/epochs/10)
warmup_steps = math.ceil(train_steps * 0.1)

dataset_split = dataset_small.train_test_split(test_size=0.2)
dataset_split["test"] = dataset_split["test"].select(range(15)) # remove for better evaluation (takes longer)
total_steps = 62
print(dataset_split)

output_dir = "train_checkpoints"

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', 'text'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['article', 'summary', 'text'],
        num_rows: 15
    })
})


In [17]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_split["train"],
    eval_dataset = dataset_split["test"],
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 12, # adjust if you have less/more cores in ur cpu
    packing = False,
    args = UnslothTrainingArguments(
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = eval_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        warmup_steps = warmup_steps,
        #max_steps = train_steps,
        eval_steps = 30,
        save_steps = 10,
        eval_strategy = "steps",
        save_strategy = "steps",
        learning_rate = 0.0001, # smaller learning rate could be better
        fp16 = False,
        bf16 = True,
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.005,
        lr_scheduler_type = "cosine",
        seed = 3047,
        output_dir = output_dir
    )
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1600 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/15 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


lez send it

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,600 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 16 x 1) = 64
 "-____-"     Trainable parameters = 24,313,856 of 3,237,066,752 (0.75% trained)


Step,Training Loss,Validation Loss


In [ ]:
from unsloth import FastLanguageModel
import torch

# Try new model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "content/train_checkpoints/checkpoint-120",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128257, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [3]:
test_article = """Suomalaistaustaisen startup-yrityksen Donut Labin uusi akkuteknologia voi muuttaa radikaalisti kilpailuasetelmia maailman akkumarkkinoilla, jos yrityksen hurjat väitteet uuden akun ominaisuuksista pitävät paikkaansa.

Nykyään Kiina on selvässä johtoasemassa akkuteknologian kehittämisessä ja akkujen valmistuksessa.

Lappeenrannan-Lahden teknillisen yliopiston sähkötekniikan professorin Pertti Kaurasen mukaan Donut Labin Yhdysvaltain CES-messuilla esittelemä uusi kiinteän olomuodon akkuteknologia voisi muuttaa tilannetta Euroopan ja Suomen hyväksi.

– Tällainen teknologia varmasti kelpaisi eurooppalaisille autonvalmistajille. Ne saisivat sen avulla kilpailuetua etenkin kiinalaisiin sähköautonvalmistajiin, professori Kauranen sanoo.

LUT-yliopiston sähkötekniikan professori Pertti Kauranen kuvattuna yliopiston sisätiloissa. 

Avaa kuvien katselu
Professori Pertti Kaurasen mielestä akkuteollisuuden voimatasopaino voi järkkyä, jos Donut Labin akkuteknologia lyö läpi. Kuva: Kalle Schönberg / Yle
Myös Kiinan hallitseva asema nykyisen akkuteknologian kriittisten raaka-aineiden, kuten litiumin, louhinnassa ja jalostamisessa voisi horjua, koska Donut Labin mukaan sen akun raaka-aineet ovat yleisesti saatavissa.

– Pääsisimme eroon geopoliittisista riippuvuuksista, jos raaka-aineita olisi saatavissa helposti mistä tahansa, Kauranen sanoo.

Hurjat ominaisuudet
Donut Lab esitteli akkuteknolgiaansa tammikuun alussa Yhdysvalloissa Las Vegasin CES-messuilla. Yrityksen mukaan kyse on maailman ensimmäisestä teolliseen tuotantoon valmiista kiinteän olomuodon akusta.

Donut Labin mukaan sen akkuja asennetaan jo virolaisen Verge Motorcyclesin valmistamiin sähkömoottoripyöriin, joista ensimmäiset lähtevät asiakkaille lähikuukausien aikana.

Ylen tietojen mukaan Verge-moottoripyörän valmistaja on kuitenkin ajautunut talouskriisiin.

Donut Labin mukaan heidän akkunsa energiatiheys on 400 Wh/kg, se latautuu täyteen alle viidessä minuutissa ja se kestää jopa 100 000 lataussykliä.

Yhtiön mukaan akku toimisi lähes täydellä teholla vielä 30 asteen pakkasessa ja yli sadan asteen kuumuudessa. Lisäksi akut olisi edullista valmistaa ja raaka-aineet saataisiin läheltä. Akut olisivat myös paloturvallisia ja ekologisia.

Sähkötekniikan professori Pertti Kauranen kertoo videolla, että näillä ominaisuuksilla suomalaisakku ohittaisi kirkkaasti nykyiset litiumakut.

LUT-yliopiston sähkötekniikan professori Pertti Kaurasen mukaan suomalaisakun väitetyt ominaisuudet ovat huomattavasti paremmat kuin parhaimpien nykyakkujen ominaisuudet.Video: Kalle Schönberg/Yle
Kiinasta tyrmäys väitteille
Donut Labin kovat väitteet akkuteknologiastaan ovat herättäneet epäilyjä asiantuntijapiireissä.

Asiaa on kommentoitu myös Kiinasta. Maailman suurimpiin kuuluvan kiinalaisen akkuvalmistajan Svoltin toimitusjohtaja Yang Hongxin tyrmäsi kovin sanoin Donut Labin väitteet kiinalaisten tiedotusvälineiden haastattelussa. Hongxinin mukaan kyse on huijauksesta.

Myös Aalto-yliopiston akkukemian ja -materiaalien professori Tanja Kallio epäilee väitteiden paikkaansapitävyyttä.

– Se tuntuu ihan uskomattomalta. Se rikkoisi kaikkia niitä fysiikan ja kemian periaatteita, joita itse tunnen, Kallio sanoo.

Teknologia tuntematonta
Donut Lab ei ole ilmoittanut, minkälaista teknologiaa se käyttää uudessa akussaan. Yhden vihjeen teknologian mahdollisesta alkuperästä antaa Donut Labin osakkuusyrityksen Nordic Nano Groupin antamat tiedot.

Imatralla tehdasta käynnistävä Nordic Nano Group on kertonut, että se pystyy valmistamaan akkukennoja nanomassasta, joka on kehitetty saksalaisen tutkimuksen pohjalta.

Yrityksen toimitilat lumisessa maisemassa aamuhämärässä Imatralla.

Avaa kuvien katselu
Nordic Nano Group on käynnistämässä tehdasta Imatralla. Kuva: Tanja Hannus / Yle
Nordic Nano Groupin toimitusjohtaja Esa Parjanen kertoi Ylelle vuoden 2024 lokakuussa, että nanomassa voi korvata autojen ja kännyköiden akuissa tyypillisesti käytetyn litiumin.

– Nämä nanoakkukennot kestävät kymmeniätuhansia latauskertoja ja niihin mahtuu enemmän energiaa. Ne ovat myös paloturvallisia, eivätkä voi räjähtää, Parjanen sanoi.

Yrityksen mukaan se alkaa tuottamaan samasta aineesta Imatran-tehtaassa aurinkopaneeleja ja aurinkoenegiaa kerääviä ohutkalvopinnoitteita jo ennen kesää. Varsinainen massatuotanto alkaa syksyllä.

Parjanen mukaan tehtaassa ei tuotettaisi akkukennoja. Hän ei kerro, missä niiden tuotanto mahdollisesti alkaa.
"""

In [15]:
# same format as training
messages = [
    {
        "role": "user", 
        # MUST match the training formatting perfectly!
        "content": f"Tiivistä seuraava uutinen selkosuomeksi:\n\n{test_article}\n"
    }
]

# tokenize with the "Generation Prompt"
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # so model knows to start answering
    return_tensors = "pt",
).to("cuda")

# tells the model that every single token is important and not to ignore anything
attention_mask = torch.ones_like(inputs)

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 512,
    use_cache = True,
    temperature = 0.4, 
)

response = tokenizer.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]

print("--- SUMMARY ---")
print(response)

--- SUMMARY ---
Suomalainen startup-yritys Donut Lab on esittänyt uuden akkuteknologian. Yritys kertoi, että sen akku on kiinteän olomuodon akku. Kyse on maailman ensimmäisestä teollisesta tuotantoon valmiasta kiinteän olomuodon akusta. Donut Labin akku on 400 Wh/kg, se latautuu täyteen alle viidessä minuutissa. Akku kestää jopa 100 000 lataussykliä. Donut Labin akku toimii lähes täydellä teholla 30 asteen pakkasessa ja yli sadan asteen kuumuudessa. Akku on myös paloturvallinen ja ekologinen. Donut Labin akku on parempi kuin nykyiset akut. Donut Labin akku on 400 Wh/kg, nykyiset akut ovat 150 Wh/kg.


In [2]:
from unsloth import FastLanguageModel
import torch

# Try old model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNo

In [7]:
# same format as training
messages = [
    {
        "role": "user", 
        # MUST match the training formatting perfectly!
        "content": f"Tiivistä seuraava uutinen selkosuomeksi:\n\n{test_article}\n"
    }
]

# tokenize with the "Generation Prompt"
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # so model knows to start answering
    return_tensors = "pt",
).to("cuda")

# tells the model that every single token is important and not to ignore anything
attention_mask = torch.ones_like(inputs)

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 512,
    use_cache = True,
)

response = tokenizer.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]

print("--- SUMMARY ---")
print(response)

--- SUMMARY ---
Suomalaista startup-yritys Donut Labin uusi akkuteknologia voisi muuttaa radikaalisti kilpailuasetelmia maailman akkumarkkinoilla. Yrityksen mukaan kyse on maailman ensimmäisestä teolliseen tuotantoon valmiista kiinteän olomuodon akusta.

Donut Labin akkuteknologia on herättänyt epäilyjä asiantuntijapiireissä, mutta yrityksen mukaan sen akkuja asennetaan jo virolaisen Verge Motorcyclesin valmistamiin sähkömoottoripyöriin.

Verge-moottoripyörän valmistaja on kuitenkin ajautunut talouskriisiin, ja yrityksen mukaan heidän akkunsa energiatiheys on 400 Wh/kg, se latautuu täyteen alle viidessä minuutissa ja se kestää jopa 100 000 lataussykliä.

Sähkötekniikan professori Pertti Kauranen kertoo videolla, että näillä ominaisuuksilla suomalaisakku ohittaisi kirkkaasti nykyiset litiumakut.

Kiinasta tyrmäys väitteille on kommentoitu myös. Maailman suurimpiin kuuluvan kiinalaisen akkuvalmistajan Svoltin toimitusjohtaja Yang Hongxin tyrmäsi on herättänyt epäilyjä. Hongxinin mukaan k